In [16]:
import torch
 
a = torch.tensor([1, 2, 3.], requires_grad=True)
print(a)
out = a.tanh()
print(out)
c = out.detach()  # 需要走注意的是，通过.detach() “分离”得到的的变量会和原来的变量共用同样的数据，而且新分离得到的张量是不可求导的，c发生了变化，原来的张量也会发生变化
c.zero_()  # 改变c的值，原来的out也会改变
print(c.requires_grad)
print(c)
print(out.requires_grad)
print(out , out.grad)
print("----------------------------------------------")
 
out.sum().backward()  # 对原来的out求导，
print(a.grad)  # 此时会报错，错误结果参考下面,显示梯度计算所需要的张量已经被“原位操作inplace”所更改了。


tensor([1., 2., 3.], requires_grad=True)
tensor([0.7616, 0.9640, 0.9951], grad_fn=<TanhBackward>)
False
tensor([0., 0., 0.])
True
tensor([0., 0., 0.], grad_fn=<TanhBackward>) None
----------------------------------------------


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [3]], which is output 0 of TanhBackward, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [7]:

 
import torch
 
a = torch.tensor([1, 2, 3.], requires_grad=True)
print(a)
out = a.tanh()
print(out)
c = out.data  # 需要走注意的是，通过.data “分离”得到的的变量会和原来的变量共用同样的数据，而且新分离得到的张量是不可求导的，c发生了变化，原来的张量也会发生变化
c.zero_() # 改变c的值，原来的out也会改变
print(c.requires_grad)
print(c)
print(out.requires_grad)
print(out)
print("----------------------------------------------")
 
out.sum().backward()  # 对原来的out求导，
print(a.grad)  # 不会报错，但是结果却并不正确
 

tensor([1., 2., 3.], requires_grad=True)
tensor([0.7616, 0.9640, 0.9951], grad_fn=<TanhBackward>)
False
tensor([0., 0., 0.])
True
tensor([0., 0., 0.], grad_fn=<TanhBackward>)
----------------------------------------------
tensor([1., 1., 1.])


*总之，data和detach()都共用同一个数据，无论是data数据还是detach()数据对应的变量改变时都会导致原来的变量改变。但是dat分离的变量in-place的时候不会被autograd追踪，那么如果用错误的tensor去求导的话会得到一个错误的值，这样做是非常危险的。使用detach的话当我做in-place的时候会被autograd追踪到，进行backward的时候就会报错。*